In [1]:
from notebook_utils import *

In [2]:
experiments_ids = [BitflipExperimentID.IPMA2, PhaseflipExperimentID.IPMA, ResetExperimentID.main]

In [11]:
mc_guarantees_file_name = "mc_guarantees_.csv"
counts_dicts = dict() # horizon -> List[hardware_specs]
all_lambdas = load_all_lambdas(ResetExperimentID.main)
for experiment_id in [ResetExperimentID.main]:
    file_path = os.path.join(get_project_path(), get_experiment_name_path(experiment_id), experiment_id.value, mc_guarantees_file_name)
    f = open(file_path)
    lines = f.readlines()[1:]
    f.close()
    for line in lines:
        elements = line.split(",")
        hardware = elements[0]
        embedding_index = int(elements[1])
        horizon = int(elements[2])
        my_guarantee = float(elements[3])
        trad_guarantee = float(elements[4])
        best_trad_guarantee = float(elements[6])
        curr_lambda = all_lambdas[find_enum_object(hardware, HardwareSpec)][embedding_index][horizon]
        if curr_lambda >= 0.999 and (best_trad_guarantee < 0.999):
            if horizon not in counts_dicts.keys():
                counts_dicts[horizon] = []
            counts_dicts[horizon].append((hardware, embedding_index, best_trad_guarantee, all_lambdas[find_enum_object(hardware, HardwareSpec)][embedding_index][horizon]))
counts_dicts

{6: [('fake_quito', 1, 0.997, 0.9991861628175757),
  ('fake_sydney', 0, 0.997, 0.9994844904373222),
  ('fake_melbourne', 4, 0.997, 0.9992838536063355)],
 7: [('fake_quito', 1, 0.998, 0.9995534320584215),
  ('fake_almaden', 4, 0.993, 0.9991993807113562),
  ('fake_sydney', 0, 0.998, 0.9997020534858817),
  ('fake_melbourne', 4, 0.998, 0.9994932409673649)]}

In [10]:
target_embeddings = [(HardwareSpec.QUITO, 1), (HardwareSpec.SYDNEY, 0), (HardwareSpec.MELBOURNE, 4), (HardwareSpec.ALMADEN, 4)]

embeddings = load_all_embeddings(ResetExperimentID.main, False)
for (hardware_spec, embedding_index) in target_embeddings:
    noise_model = NoiseModel(hardware_spec, False)
    qubit_index = embeddings[hardware_spec][embedding_index][0]
    instruction = Instruction(qubit_index, Op.MEAS)
    print(noise_model.instructions_to_channel[instruction].get_ind_probability(0,0), noise_model.instructions_to_channel[instruction].get_ind_probability(1,1))


0.9892 0.9208
0.9924 0.9302
0.9912 0.9136
0.9933333333333333 0.8533333333333334


In [20]:
for (horizon, hardware_specs) in counts_dicts.items():
    print(horizon, len(hardware_specs))

7 14
6 7
5 9


In [34]:
experiment_id = GHZExperimentID.EMBED
file_path = os.path.join(get_project_path(), get_experiment_name_path(experiment_id),"guarantees_embed.csv")
f = open(file_path)
lines = f.readlines()[1:]
f.close()
for line in lines:
    elements = line.split(",")
    hardware = elements[0]
    my_guarantee = float(elements[1])
    trad_guarantee = float(elements[2])
    diff = float(elements[3])
    if my_guarantee >= 0.99 and (my_guarantee != trad_guarantee):
        print(hardware,my_guarantee, trad_guarantee, diff)



fake_hanoi 0.99 0.988 0.002
fake_auckland 0.99 0.979 0.011
fake_geneva 0.99 0.982 0.008


# Error in single qubit gates

In [2]:
error_rates = []
for hardware_spec in HardwareSpec:
    noise_model = NoiseModel(hardware_spec, thermal_relaxation=False)
    
    for (instruction, channel) in noise_model.instructions_to_channel.items():
        assert isinstance(instruction, Instruction)
        
        if (not is_multiqubit_gate(instruction.op)):
            if  not instruction.is_meas_instruction():
                assert isinstance(channel, QuantumChannel)
                error_rates.append(channel.estimated_success_prob)
                
df = pd.DataFrame.from_dict({
            'error_rates': error_rates,
        }) 

In [3]:
df.describe()

,error_rates
count,4232.000000
mean,0.999150
std,0.002319
min,0.959810
25%,0.999291
50%,0.999659
75%,1.000000
max,1.000000
